# Секреты темнолесья. Предобработка данных

- Автор: Мулашкина Татьяна
- Дата: 19.03.2025

### Цели и задачи проекта

<b>Цель:</b> изучить развитие игровой индустрии с 2000 по 2013 год. <br> 
<b>Задачи:</b>  <br> 
    1) Познакомиться с данными <br> 
    2) Проверить данные на корректность <br> 
    3) Провести предобработку: получить необходимый срез данных для последующего анализа <br> 

### Описание данных

Данные /datasets/new_games.csv содержат информацию о продажах игр разных жанров и платформ, а также пользовательские и экспертные оценки игр: <br>
- Name — название игры.
- Platform — название платформы.
- Year of Release — год выпуска игры.
- Genre — жанр игры.
- NA sales — продажи в Северной Америке (в миллионах проданных копий).
- EU sales — продажи в Европе (в миллионах проданных копий).
- JP sales — продажи в Японии (в миллионах проданных копий).
- Other sales — продажи в других странах (в миллионах проданных копий).
- Critic Score — оценка критиков (от 0 до 100).
- User Score — оценка пользователей (от 0 до 10).
- Rating — рейтинг организации ESRB (англ. Entertainment Software Rating Board). Эта ассоциация определяет рейтинг компьютерных игр и присваивает им подходящую возрастную категорию.

### Содержимое проекта

1. Загрузка данных и знакомство с ними
2. Предобработка данных: 
- исправления типов данных
- работа с пропусками
- работа с дубликатами
3. Фильтрация данных
4. Категоризация данных

---

## 1. Загрузка данных и знакомство с ними

- Загрузите необходимые библиотеки Python и данные датасета `/datasets/new_games.csv`.


In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('/datasets/new_games.csv')

- Познакомьтесь с данными: выведите первые строки и результат метода `info()`.


In [4]:
df.head()

,Name,Platform,Year of Release,Genre,NA sales,EU sales,JP sales,Other sales,Critic Score,User Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16954 non-null  object 
 1   Platform         16956 non-null  object 
 2   Year of Release  16681 non-null  float64
 3   Genre            16954 non-null  object 
 4   NA sales         16956 non-null  float64
 5   EU sales         16956 non-null  object 
 6   JP sales         16956 non-null  object 
 7   Other sales      16956 non-null  float64
 8   Critic Score     8242 non-null   float64
 9   User Score       10152 non-null  object 
 10  Rating           10085 non-null  object 
dtypes: float64(4), object(7)
memory usage: 1.4+ MB


- Названия столбцов совпадают с представленным описанием. Однако содержат пробелы (возможно стоит заменить на нижнее подчеркивание) <br>
- Год выпуска игры (Year of Release) приден к типу данных float64. <br>
- Данные о продажах в Европе и Японии (EU sales и JP sales),а также оценка пользователей (User Score) представлены в типе данных object, а не float64 <br>
- Остальные данные соответсвуют описанию. <br>
- Всего строк в датасете 16956. <br>
- Наибольшее количество пропусков (около половины данных) наблюдается для данных об оценке критиков (Critic Score) <br>
- Также пропуски наблюдаются в колонках Name, Year of Release, Genre, User Score, Rating <br>
- Количество пропусков в колонках Name и Genre не значительно (меньше 1 процента) и в колонке Year of Release (меньше 2 процентов) <br>
- В колонках User Score и Rating также содержится большое количество пропусков

---

## 2.  Проверка ошибок в данных и их предобработка


### 2.1. Названия, или метки, столбцов датафрейма

- Выведите на экран названия всех столбцов датафрейма и проверьте их стиль написания.
- Приведите все столбцы к стилю snake case. Названия должны быть в нижнем регистре, а вместо пробелов — подчёркивания.

In [6]:
df.columns

Index(['Name', 'Platform', 'Year of Release', 'Genre', 'NA sales', 'EU sales',
       'JP sales', 'Other sales', 'Critic Score', 'User Score', 'Rating'],
      dtype='object')

In [7]:
df.columns = df.columns.str.replace(' ', '_').str.lower()
df.columns

Index(['name', 'platform', 'year_of_release', 'genre', 'na_sales', 'eu_sales',
       'jp_sales', 'other_sales', 'critic_score', 'user_score', 'rating'],
      dtype='object')

### 2.2. Типы данных

- Если встречаются некорректные типы данных, предположите их причины.
- При необходимости проведите преобразование типов данных. Помните, что столбцы с числовыми данными и пропусками нельзя преобразовать к типу `int64`. Сначала вам понадобится обработать пропуски, а затем преобразовать типы данных.

Некорректные типы данных встречаются для колонок: year_of_release, eu_sales, jp_sales, user_score. <br>
Возможные причины: <br>
- Год выпуска игры (year_of_release) был сохранен в типе данных float64, возможно измение типа данны хпроизошло из-за некорректного переноса данных
- Возможно колонки eu_sales, jp_sales, user_score содержат не только числа, но и строки, поэтому сохранены в формате object

In [8]:
df['year_of_release'] = pd.to_numeric(df['year_of_release'].fillna(0), downcast='integer', errors='coerce')
df[['eu_sales', 'jp_sales', 'user_score']] = df[['eu_sales', 'jp_sales', 'user_score']].apply(pd.to_numeric, errors = 'coerce')

- В числовых столбцах могут встретиться строковые значения, например `unknown` или другие. Приводите такие столбцы к числовому типу данных, заменив строковые значения на пропуски.

In [9]:
df.head()

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,Wii Sports,Wii,2006,Sports,41.36,28.96,3.77,8.45,76.0,8.0,E
1,Super Mario Bros.,NES,1985,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009,Sports,15.61,10.93,3.28,2.95,80.0,8.0,E
4,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


### 2.3. Наличие пропусков в данных

- Посчитайте количество пропусков в каждом столбце в абсолютных и относительных значениях.


In [10]:
print('Количество пропусков в каждом стобце в абсолютных значениях')
print(df.isna().sum())
print('Количество пропусков в каждом стобце в долях')
print(df.isna().mean())

Количество пропусков в каждом стобце в абсолютных значениях
name                  2
platform              0
year_of_release       0
genre                 2
na_sales              0
eu_sales              6
jp_sales              4
other_sales           0
critic_score       8714
user_score         9268
rating             6871
dtype: int64
Количество пропусков в каждом стобце в долях
name               0.000118
platform           0.000000
year_of_release    0.000000
genre              0.000118
na_sales           0.000000
eu_sales           0.000354
jp_sales           0.000236
other_sales        0.000000
critic_score       0.513918
user_score         0.546591
rating             0.405225
dtype: float64


- Изучите данные с пропущенными значениями. Напишите промежуточный вывод: для каких столбцов характерны пропуски и сколько их. Предположите, почему пропуски могли возникнуть. Укажите, какие действия с этими данными можно сделать и почему.


- Наибольшее количество пропусков встречается в колонках critic_score (51%) и user_score (55%). Данные в этих колонках относят к оценке игры критиков и пользователей. Такое большое количество пропусков может быть связано с тем, что не все игры оценивают. Возможно, если игра непопулярна, то пользователи и критики не обратили на нее внимание и не оценивали.
- Также большое количество пропусков (40%) наблюдается в колонке rating, которая сожержит рейтинг организации ESRB. Это может быть связано  с тем, что не все игры оценивались организацией по причинам: некоторые разработчики могут не подавать свои игры на оценку, независимые разработчики могут не иметь ресурсов для прохождения процесса оценки, невозможно оценить весь пользовательский контент в онлайн-играх. А также игры, выпущенные до создания ESRB, могут не иметь рейтинга.
- Кроме того пропуски содержатся и в других колонках (name, year_of_release, genre, eu_sales, jp_sales), что может быть связано просто с ошибкой в сборе данных или отсутсвием данной информации для некоторых игр. Однако количество таких пропусков невелико и составляет меньше 2%.

- Обработайте пропущенные значения. Для каждого случая вы можете выбрать оптимальный, на ваш взгляд, вариант: заменить на определённое значение, оставить как есть или удалить.
- Если вы решите заменить пропуски на значение-индикатор, то убедитесь, что предложенное значение не может быть использовано в данных.
- Если вы нашли пропуски в данных с количеством проданных копий игры в том или ином регионе, их можно заменить на среднее значение в зависимости от названия платформы и года выхода игры.

In [11]:
print(df[df['name'].isna()])

      name platform  year_of_release genre  na_sales  eu_sales  jp_sales  \
661    NaN      GEN             1993   NaN      1.78      0.53      0.00   
14439  NaN      GEN             1993   NaN      0.00      0.00      0.03   

       other_sales  critic_score  user_score rating  
661           0.08           NaN         NaN    NaN  
14439         0.00           NaN         NaN    NaN  


Колонки name и genre содержат по 2 пропуска, причем в одних и тех же строках. Поэтому эти данные можно просто удалить.

In [12]:
df = df.dropna(subset = ['name'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16954 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16954 non-null  object 
 1   platform         16954 non-null  object 
 2   year_of_release  16954 non-null  int16  
 3   genre            16954 non-null  object 
 4   na_sales         16954 non-null  float64
 5   eu_sales         16948 non-null  float64
 6   jp_sales         16950 non-null  float64
 7   other_sales      16954 non-null  float64
 8   critic_score     8242 non-null   float64
 9   user_score       7688 non-null   float64
 10  rating           10085 non-null  object 
dtypes: float64(6), int16(1), object(4)
memory usage: 1.5+ MB


Пропуски в данных с количеством проданных копий игры в Европе (eu_sales) и Японии (jp_sales) заменим на среднее значение в зависимости от названия платформы и года выхода игры.

In [13]:
def mean_group_year_eu(row):
    if (pd.isna(row['eu_sales'])):
        group = df[(df['year_of_release'] == row['year_of_release']) & (df['platform'] == row['platform'])]
        return group['eu_sales'].mean()
    else:
        return row['eu_sales']
    
df['eu_sales'] = df.apply(mean_group_year_eu, axis=1)

In [14]:
def mean_group_year_jp(row):
    if (pd.isna(row['jp_sales'])):
        group = df[(df['year_of_release'] == row['year_of_release']) & (df['platform'] == row['platform'])]
        return group['jp_sales'].mean()
    else:
        return row['jp_sales']
    
df['jp_sales'] = df.apply(mean_group_year_jp, axis=1)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16954 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16954 non-null  object 
 1   platform         16954 non-null  object 
 2   year_of_release  16954 non-null  int16  
 3   genre            16954 non-null  object 
 4   na_sales         16954 non-null  float64
 5   eu_sales         16954 non-null  float64
 6   jp_sales         16954 non-null  float64
 7   other_sales      16954 non-null  float64
 8   critic_score     8242 non-null   float64
 9   user_score       7688 non-null   float64
 10  rating           10085 non-null  object 
dtypes: float64(6), int16(1), object(4)
memory usage: 1.5+ MB


Для заполнения пропусков в колонках с оценками критиков и пользователей (critic_score и  user_score) воспользуемся значением индикатором -1. Пропуски в колонке rating оставим пустыми.

In [16]:
df[['critic_score', 'user_score']] = df[['critic_score', 'user_score']].fillna(-1)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16954 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16954 non-null  object 
 1   platform         16954 non-null  object 
 2   year_of_release  16954 non-null  int16  
 3   genre            16954 non-null  object 
 4   na_sales         16954 non-null  float64
 5   eu_sales         16954 non-null  float64
 6   jp_sales         16954 non-null  float64
 7   other_sales      16954 non-null  float64
 8   critic_score     16954 non-null  float64
 9   user_score       16954 non-null  float64
 10  rating           10085 non-null  object 
dtypes: float64(6), int16(1), object(4)
memory usage: 1.5+ MB


### 2.4. Явные и неявные дубликаты в данных

- Изучите уникальные значения в категориальных данных, например с названиями жанра игры, платформы, рейтинга и года выпуска. Проверьте, встречаются ли среди данных неявные дубликаты, связанные с опечатками или разным способом написания.
- При необходимости проведите нормализацию данных с текстовыми значениями. Названия или жанры игр можно привести к нижнему регистру, а названия рейтинга — к верхнему.

In [18]:
print('Количество уникальных значений для названия игры:')
print(len(df['name'].unique()))
print('Уникальные значения для жанров игры:')
print(df['genre'].unique())
print('Уникальные значения для платформ:')
print(df['platform'].unique())
print('Уникальные значения для рейтинга:')
print(df['rating'].unique())
print('Уникальные значения для года выпуска:')
print(df['year_of_release'].unique())

Количество уникальных значений для названия игры:
11559
Уникальные значения для жанров игры:
['Sports' 'Platform' 'Racing' 'Role-Playing' 'Puzzle' 'Misc' 'Shooter'
 'Simulation' 'Action' 'Fighting' 'Adventure' 'Strategy' 'MISC'
 'ROLE-PLAYING' 'RACING' 'ACTION' 'SHOOTER' 'FIGHTING' 'SPORTS' 'PLATFORM'
 'ADVENTURE' 'SIMULATION' 'PUZZLE' 'STRATEGY']
Уникальные значения для платформ:
['Wii' 'NES' 'GB' 'DS' 'X360' 'PS3' 'PS2' 'SNES' 'GBA' 'PS4' '3DS' 'N64'
 'PS' 'XB' 'PC' '2600' 'PSP' 'XOne' 'WiiU' 'GC' 'GEN' 'DC' 'PSV' 'SAT'
 'SCD' 'WS' 'NG' 'TG16' '3DO' 'GG' 'PCFX']
Уникальные значения для рейтинга:
['E' nan 'M' 'T' 'E10+' 'K-A' 'AO' 'EC' 'RP']
Уникальные значения для года выпуска:
[2006 1985 2008 2009 1996 1989 1984 2005 1999 2007 2010 2013 2004 1990
 1988 2002 2001 2011 1998 2015 2012 2014 1992 1997 1993 1994 1982 2016
 2003 1986 2000    0 1995 1991 1981 1987 1980 1983]


- Данные о платформах не содержат неявных дубликатов
- Видно, что данные о жанре игры (genre) содержат неявные дубликаты, связанные с разным написанием. Приведем все названия жанров к нижнем регистру.
- Название игр также приведем к нижнему регистру
- Данные о годе выпуска содержат значения 0, которые появилось на месте пропусков

In [19]:
df['genre'] = df['genre'].str.lower()
df['name'] = df['name'].str.lower()
print('Количество уникальных значений для названия игры:')
print(len(df['name'].unique()))

Количество уникальных значений для названия игры:
11559


В данных о возрастном рейтинге игры было обнаружено значения 'K-A'. <br>
По информации из википедии: "«E» («Everyone») — «Для всех»: Содержание вполне подходит для всех возрастов (ранее с 6 лет). Такие игры могут понравиться и взрослым. Игры с этим рейтингом могут содержать минимальное насилие, в основном «мультяшного» характера. Первоначально "K-A" ("Kids to Adults")" <br>
Таким образом рейтинг 'K-A' это тоже самое рейтинг 'E'. <br>
Заменим 'K-A' на 'E':

In [20]:
df['rating'] = df['rating'].str.replace('K-A', 'E')
print(df['rating'].unique())

['E' nan 'M' 'T' 'E10+' 'AO' 'EC' 'RP']


- После того как нормализуете данные и устраните неявные дубликаты, проверьте наличие явных дубликатов в данных.

In [21]:
print('Количество явных дубликатов:')
df.duplicated().sum()

Количество явных дубликатов:


241

- Напишите промежуточный вывод: укажите количество найденных дубликатов и действия по их обработке.

Количество явных дубликатов составляет 241. Удалим строки с дублирующимися данными:

In [22]:
df_cleaned = df.drop_duplicates(subset=None, keep='first', inplace=False) 
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16713 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16713 non-null  object 
 1   platform         16713 non-null  object 
 2   year_of_release  16713 non-null  int16  
 3   genre            16713 non-null  object 
 4   na_sales         16713 non-null  float64
 5   eu_sales         16713 non-null  float64
 6   jp_sales         16713 non-null  float64
 7   other_sales      16713 non-null  float64
 8   critic_score     16713 non-null  float64
 9   user_score       16713 non-null  float64
 10  rating           9949 non-null   object 
dtypes: float64(6), int16(1), object(4)
memory usage: 1.4+ MB


- В процессе подготовки данных вы могли что-либо удалять, например строки с пропусками или ошибками, дубликаты и прочее. В этом случае посчитайте количество удалённых строк в абсолютном и относительном значениях.

In [23]:
print('Изначально количество строк в датасете было: 16956 строк')
print('После удаления строк с пропусками осталось: 16954 строк')
print('Было обнаружено 241 явных дубликатов, после удаления которых, осталось: 16713 строк')
count_deleted_str = 16956 - 16713
per_deleted_str = round(count_deleted_str / 16956, 4)
print(f'Общее количество удаленных строк: {count_deleted_str}')
print(f'Доля удаленных строк: {per_deleted_str}')

Изначально количество строк в датасете было: 16956 строк
После удаления строк с пропусками осталось: 16954 строк
Было обнаружено 241 явных дубликатов, после удаления которых, осталось: 16713 строк
Общее количество удаленных строк: 243
Доля удаленных строк: 0.0143


- После проведения предобработки данных напишите общий промежуточный вывод.

- В результате предобработки были приведены к правильному типу даннных данные в колонках: year_of_release, eu_sales, jp_sales и user_score
- Были обнаружен пропуски данных, которые были либо удалены (в колонках name, genre),  либо заменены на среднее значение (для eu_sales, jp_sales) или индикатор -1 (дляcritic_score, user_score) и 0 (для year_of_release),  оставлены пустыми (rating). 
- Были обнаружены неявные дубликаты в genre, которые появились из-за разного написания,  а также было обнаружено разное написание одного и того же рейтинга в rating. Данные неявные дубликаты были устранены.
- А также были обнаружены и удалены 241 явных дубликатов

---

## 3. Фильтрация данных

Коллеги хотят изучить историю продаж игр в начале XXI века, и их интересует период с 2000 по 2013 год включительно. Отберите данные по этому показателю. Сохраните новый срез данных в отдельном датафрейме, например `df_actual`.

In [24]:
df_actual = df_cleaned[(df_cleaned['year_of_release'] >= 2000) & (df_cleaned['year_of_release'] <= 2013)]

In [25]:
df_actual.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12781 entries, 0 to 16954
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             12781 non-null  object 
 1   platform         12781 non-null  object 
 2   year_of_release  12781 non-null  int16  
 3   genre            12781 non-null  object 
 4   na_sales         12781 non-null  float64
 5   eu_sales         12781 non-null  float64
 6   jp_sales         12781 non-null  float64
 7   other_sales      12781 non-null  float64
 8   critic_score     12781 non-null  float64
 9   user_score       12781 non-null  float64
 10  rating           8723 non-null   object 
dtypes: float64(6), int16(1), object(4)
memory usage: 1.1+ MB


---

## 4. Категоризация данных
    
Проведите категоризацию данных:
- Разделите все игры по оценкам пользователей и выделите такие категории: высокая оценка (от 8 до 10 включительно), средняя оценка (от 3 до 8, не включая правую границу интервала) и низкая оценка (от 0 до 3, не включая правую границу интервала).

In [26]:
df_actual['user_score_group'] = pd.cut(df_actual['user_score'], bins=[0, 2, 7, 10], labels=["низкая оценка", "средняя оценка", "высокая оценка"])
df_actual.head()

/tmp/ipykernel_88/4162377342.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_actual['user_score_group'] = pd.cut(df_actual['user_score'], bins=[0, 2, 7, 10], labels=["низкая оценка", "средняя оценка", "высокая оценка"])


,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,user_score_group
0,wii sports,Wii,2006,sports,41.36,28.96,3.77,8.45,76.0,8.0,E,высокая оценка
2,mario kart wii,Wii,2008,racing,15.68,12.76,3.79,3.29,82.0,8.3,E,высокая оценка
3,wii sports resort,Wii,2009,sports,15.61,10.93,3.28,2.95,80.0,8.0,E,высокая оценка
6,new super mario bros.,DS,2006,platform,11.28,9.14,6.50,2.88,89.0,8.5,E,высокая оценка
7,wii play,Wii,2006,misc,13.96,9.18,2.93,2.84,58.0,6.6,E,средняя оценка


- Разделите все игры по оценкам критиков и выделите такие категории: высокая оценка (от 80 до 100 включительно), средняя оценка (от 30 до 80, не включая правую границу интервала) и низкая оценка (от 0 до 30, не включая правую границу интервала).

In [27]:
df_actual['critic_score_group'] = pd.cut(df_actual['critic_score'], bins=[0, 29, 79, 100], labels=["низкая оценка", "средняя оценка", "высокая оценка"])
df_actual.head()

/tmp/ipykernel_88/338164334.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_actual['critic_score_group'] = pd.cut(df_actual['critic_score'], bins=[0, 29, 79, 100], labels=["низкая оценка", "средняя оценка", "высокая оценка"])


,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,user_score_group,critic_score_group
0,wii sports,Wii,2006,sports,41.36,28.96,3.77,8.45,76.0,8.0,E,высокая оценка,средняя оценка
2,mario kart wii,Wii,2008,racing,15.68,12.76,3.79,3.29,82.0,8.3,E,высокая оценка,высокая оценка
3,wii sports resort,Wii,2009,sports,15.61,10.93,3.28,2.95,80.0,8.0,E,высокая оценка,высокая оценка
6,new super mario bros.,DS,2006,platform,11.28,9.14,6.50,2.88,89.0,8.5,E,высокая оценка,высокая оценка
7,wii play,Wii,2006,misc,13.96,9.18,2.93,2.84,58.0,6.6,E,средняя оценка,средняя оценка


- После категоризации данных проверьте результат: сгруппируйте данные по выделенным категориям и посчитайте количество игр в каждой категории.

In [28]:
user_score_group = df_actual.groupby('user_score_group')['name'].agg('count')
print(user_score_group)

user_score_group
низкая оценка       46
средняя оценка    2351
высокая оценка    4085
Name: name, dtype: int64


In [29]:
critic_score_group = df_actual.groupby('critic_score_group')['name'].agg('count')
print(critic_score_group)

critic_score_group
низкая оценка       55
средняя оценка    5422
высокая оценка    1692
Name: name, dtype: int64


- Выделите топ-7 платформ по количеству игр, выпущенных за весь актуальный период.

In [30]:
platform_group= df_actual.groupby('platform')['name'].agg('count')
platform_group = platform_group.sort_values(ascending = False)
platform_group.head(7)

platform
PS2     2127
DS      2120
Wii     1275
PSP     1180
X360    1121
PS3     1087
GBA      811
Name: name, dtype: int64

---

## 5. Итоговый вывод

В конце напишите основной вывод и отразите, какую работу проделали. Не забудьте указать описание среза данных и новых полей, которые добавили в исходный датасет.

1. Была выполнена предобработка данных:
- В результате предобработки были приведены к правильному типу даннных данные в колонках: year_of_release (из float64 к int), eu_sales, jp_sales и user_score (из object к float64)
- Были обнаружен пропуски данных, которые были либо удалены (в колонках name, genre),  либо заменены на среднее значение (для eu_sales, jp_sales) или индикатор -1 (для critic_score, user_score) и 0 (для year_of_release),  оставлены пустыми (rating). 
- Были обнаружены неявные дубликаты в genre, которые появились из-за разного написания,  а также было обнаружено разное написание одного и того же рейтинга в rating. Данные неявные дубликаты были устранены.
- А также были обнаружены и удалены 241 явных дубликатов
Таким образом, после предобработки было удалено 243 строки, что составляет 1% от исходного количества
2. Так как для статьи понадобится изучить развитие игровой индустрии с 2000 по 2013 год, данные были отфильтрованы по годам выпуска игр: от 2000 до 2013 включительно
3. Была проведена категоризация данных по оценкам пользователей:
- высокая оценка (от 8 до 10 включительно), 
- средняя оценка (от 3 до 8, не включая правую границу интервала),
- низкая оценка (от 0 до 3, не включая правую границу интервала)
и по оценкам критиков:
- высокая оценка (от 80 до 100 включительно)
- средняя оценка (от 30 до 80, не включая правую границу интервала)
- низкая оценка (от 0 до 30, не включая правую границу интервала)
4. Для каждой категории было определено, количество игр:
по оценкам пользователся, видно, что наибольшое количество игр относятся к категории "высокая оценка", а наименьшее к категории "низкая оценка", тогда как по оценкам критиков наибольшее количество игр относятся к категории "средняя оценка". Это говорит о том, что пользователи оценивают игры более лояльно, чем критики.
5. Были определены топ-7 платформ по количеству выпускаемых игр: PS2, DS, Wii, PSP, X360, PS3, GBA